In [1]:
import os
import logging
from datetime import datetime
import tkinter as tk
from tkinter import ttk, messagebox

import psycopg2
from psycopg2 import sql
import openpyxl
from openpyxl.styles import numbers


# =========================
# Configuración de logging
# =========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filename="inventario_app.log"
)


class InventarioApp:
    """
    Aplicación Tkinter para Toma Física de Inventario.
    - Consulta por código de barras o secuencia.
    - Actualiza/Inserta en BD.
    - Informe único por código (sin duplicados).
    - Exporta resultados a Excel.
    """

    def __init__(self, root: tk.Tk) -> None:
        self.root = root
        self.root.title("TOMA FÍSICA INVENTARIO")
        self.root.geometry("1024x768")

        # Estado UI y control de duplicados en el Treeview
        self.rows_by_barcode: dict[str, str] = {}   # {codigo_barras: iid}
        self._search_after_id: str | None = None    # para debounce
        self.clear_after_search: bool = True        # limpiar buscador tras consulta

        # Inicialización UI
        self._setup_estilos()
        self._setup_ui()
        self._cargar_datos_combo()
        self._setup_eventos_teclado()

    # =========================
    # Estilos
    # =========================
    def _setup_estilos(self) -> None:
        """Configurar estilos visuales de la aplicación, con fallback si no existe el tema Azure."""
        self.style = ttk.Style(self.root)
        try:
            if os.path.exists("azure.tcl"):
                self.root.tk.call("source", "azure.tcl")
                self.style.theme_use("azure-dark")
            else:
                self.style.theme_use("clam")
        except tk.TclError:
            self.style.theme_use("clam")

        self.style.configure("TLabel", font=("Segoe UI", 10))
        self.style.configure("TButton", font=("Segoe UI", 10, "bold"), padding=5)
        self.style.configure("TEntry", padding=5)
        self.style.configure("Treeview", font=("Segoe UI", 10), rowheight=24)
        self.style.configure("Title.TLabel", font=("Segoe UI", 12, "bold"))

    # =========================
    # Conexión BD
    # =========================
    def _conectar_bd(self):
        """
        Establecer conexión con la base de datos.
        Usa variables de entorno si existen; en su defecto, valores por defecto.
        """
        try:
            conn = psycopg2.connect(
                dbname=os.getenv("PG_DB", "inventario"),
                user=os.getenv("PG_USER", "postgres"),
                password=os.getenv("PG_PASS", "postgres"),
                host=os.getenv("PG_HOST", "localhost"),
                port=os.getenv("PG_PORT", "5432"),
            )
            logging.info("Conexión a BD establecida correctamente")
            return conn
        except Exception as e:
            error_msg = f"Error de conexión: {str(e)}"
            logging.error(error_msg)
            messagebox.showerror("Error de conexión", error_msg)
            return None

    # =========================
    # Utilidades de Estado / UI
    # =========================
    @staticmethod
    def _estado_to_ui(db_val: str | None) -> str:
        """Normaliza el estado desde BD a la UI. Devuelve 'BUENO'|'INSERVIBLE'|''."""
        if not db_val:
            return ""
        return str(db_val).strip().upper()

    @staticmethod
    def _estado_to_db(ui_val: str | None) -> str | None:
        """
        Normaliza el estado desde la UI a la BD (ENUM).
        UI: 'BUENO'|'INSERVIBLE'|'' → DB ENUM: 'BUENO'|'INSERVIBLE'|NULL.
        """
        if not ui_val:
            return None
        v = ui_val.strip().upper()
        return v if v in ("BUENO", "INSERVIBLE") else None

    def _set_entry_text(self, widget: ttk.Entry, value) -> None:
        """
        Inserta texto de forma segura en un Entry:
        - Convierte None a "" y el resto a str.
        - Maneja correctamente widgets en estado readonly/disabled.
        - Evita el error Tcl: 'wrong # args ... insert index text'.
        """
        text = "" if value is None else str(value)
        try:
            prev_state = widget.cget("state")
            if prev_state in ("readonly", "disabled"):
                widget.config(state="normal")
                widget.delete(0, tk.END)
                widget.insert(0, text)
                widget.config(state=prev_state)
            else:
                widget.delete(0, tk.END)
                widget.insert(0, text)
        except tk.TclError as e:
            # Log detallado para diagnóstico
            logging.exception("Fallo insert en %s con valor=%r", str(widget), value)
            raise

    # =========================
    # Carga inicial de combos
    # =========================
    def _cargar_datos_combo(self) -> None:
        """Cargar datos para los combobox desde la base de datos."""
        conn = self._conectar_bd()
        if not conn:
            return

        try:
            with conn.cursor() as cur:
                # Cédulas
                cur.execute(
                    "SELECT cedula FROM public.personal_litoteca ORDER BY cedula"
                )
                self.cedula_combo["values"] = [str(row[0]) for row in cur.fetchall()]

                # Responsables
                cur.execute(
                    "SELECT nombre FROM public.personal_litoteca ORDER BY nombre"
                )
                self.responsable_combo["values"] = [row[0] for row in cur.fetchall()]

                # Ubicaciones
                cur.execute(
                    "SELECT ubicaciones FROM public.ubicaciones_litoteca ORDER BY ubicaciones"
                )
                self.ubicacion_combo["values"] = [row[0] for row in cur.fetchall()]

            logging.info("Datos de combobox cargados correctamente")
        except Exception as e:
            error_msg = f"Error al cargar datos de combobox: {str(e)}"
            logging.error(error_msg)
            messagebox.showerror("Error", error_msg)
        finally:
            conn.close()

    # =========================
    # UI Helpers
    # =========================
    def _limpiar_campos(self) -> None:
        """Limpiar todos los campos del formulario y el panel de secuencia (mantiene estados readonly)."""
        for entry in self.all_entries:
            self._set_entry_text(entry, "")

        # Reset combos
        self.cedula_combo.set("")
        self.responsable_combo.set("")
        self.estado_combo.set("")
        self.ubicacion_combo.set("")
        self.entry_busqueda.delete(0, tk.END)
        self.entry_secuencia.delete(0, tk.END)

        # Limpiar panel de secuencia
        for widget in self.frame_seq.winfo_children():
            widget.destroy()

        self.entry_busqueda.focus()

    def _exportar_excel(self) -> None:
        """Exportar el contenido del Treeview a un archivo Excel (preserva códigos como texto)."""
        if not self.tree.get_children():
            messagebox.showwarning("Sin datos", "No hay datos para exportar.")
            return

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        archivo = f"consulta_{timestamp}.xlsx"

        try:
            wb = openpyxl.Workbook()
            ws = wb.active
            ws.title = "Resultados"

            # Encabezados
            ws.append(self.columnas)

            # Datos
            for item in self.tree.get_children():
                vals = list(self.tree.item(item)["values"])
                # Columna 0 = Código de barras → texto explícito
                vals[0] = f"'{vals[0]}"
                ws.append(vals)

            # Forzar formato de texto en la primera columna (A)
            for cell in ws["A"][1:]:
                cell.number_format = numbers.FORMAT_TEXT

            # Ajuste de ancho
            for column in ws.columns:
                max_len = 0
                col_letter = column[0].column_letter
                for cell in column:
                    try:
                        max_len = max(max_len, len(str(cell.value)))
                    except Exception:
                        pass
                ws.column_dimensions[col_letter].width = min(max_len + 2, 50)

            wb.save(archivo)
            messagebox.showinfo("Exportado", f"Archivo guardado:\n{os.path.abspath(archivo)}")
            logging.info(f"Archivo exportado: {archivo}")

        except Exception as e:
            error_msg = f"Error al exportar: {str(e)}"
            logging.error(error_msg)
            messagebox.showerror("Error", error_msg)

    # =========================
    # Búsquedas
    # =========================
    def _debounced_keyrelease(self, _event=None) -> None:
        """Debounce: evita múltiples consultas mientras se digita manualmente."""
        if self._search_after_id is not None:
            self.root.after_cancel(self._search_after_id)
        self._search_after_id = self.root.after(180, self._buscar_codigo)

    def _buscar_codigo(self, _event=None) -> None:
        """
        Buscar elemento por código de barras.
        - Soporta Enter (escáner) o debounce en KeyRelease.
        - Limpia el campo y mantiene foco para el siguiente escaneo.
        """
        codigo = self.entry_busqueda.get().strip()

        # Validación mínima (ajusta si tu escáner genera prefijos)
        if len(codigo) < 6 or not codigo.isdigit():
            return

        conn = self._conectar_bd()
        if not conn:
            return

        try:
            with conn.cursor() as cur:
                cur.execute(
                    sql.SQL(
                        """
                        SELECT
                            centro_operacion,       -- 0
                            secuencia,              -- 1
                            codigo_barras,          -- 2
                            codigo_elemento,        -- 3
                            serie,                  -- 4
                            placa,                  -- 5
                            descripcion_elemento,   -- 6
                            tipo_parte,             -- 7
                            tipo_almacenamiento,    -- 8
                            fecha_ingreso,          -- 9
                            costo,                  -- 10
                            cedula_resp,            -- 11
                            responsable_inventario, -- 12
                            estado_elemento,        -- 13 (ENUM)
                            ubicacion_2025          -- 14
                        FROM inventario_verificado
                        WHERE codigo_barras = %s
                        LIMIT 1
                        """
                    ),
                    (codigo,),
                )
                r = cur.fetchone()

                if not r:
                    messagebox.showinfo("Sin resultados", "Código no encontrado.")
                    return

                # Relleno seguro de campos
                fecha_ingreso_fmt = r[9].strftime("%Y-%m-%d") if r[9] else ""
                costo_fmt = str(r[10]) if r[10] is not None else ""

                entries_values = [
                    r[0],        # Centro Operación
                    r[1],        # Secuencia
                    r[2],        # Código Barras
                    r[3],        # Código Elemento
                    r[4],        # Serie
                    r[5],        # Placa
                    r[6],        # Descripción
                    r[7],        # Tipo Parte
                    "",          # Almacén (Tipo) -> se carga aparte
                    fecha_ingreso_fmt,  # Fecha Ingreso
                    costo_fmt    # Costo
                ]

                for widget, val in zip(self.all_entries, entries_values):
                    if widget is self.almacen_entry:
                        continue
                    self._set_entry_text(widget, val)

                # Cargar Almacén (Tipo) respetando readonly
                self._set_entry_text(self.almacen_entry, r[8] or "")

                # Combos (nota: estado UI en mayúsculas)
                self.cedula_combo.set(str(r[11]) if r[11] is not None else "")
                self.responsable_combo.set(r[12] or "")
                self.estado_combo.set(self._estado_to_ui(r[13]))
                self.ubicacion_combo.set(r[14] or "")

                # Actualizar informe ÚNICO por código (incluye Parte)
                self._actualizar_tree_unique(
                    cbarras=r[2],
                    sec=r[1],
                    ser=r[4],
                    pla=r[5],
                    desc=r[6],
                    parte=r[7],
                    est=self._estado_to_ui(r[13]),
                    ubic=r[14],
                    resp=r[12],
                    alm=r[8]
                )

                # Preparar siguiente escaneo
                if self.clear_after_search:
                    self.entry_busqueda.delete(0, tk.END)
                    self.entry_busqueda.focus()

                logging.info(f"Búsqueda exitosa para código: {codigo}")

        except Exception as e:
            error_msg = f"Error en búsqueda: {str(e)}"
            logging.error(error_msg)
            messagebox.showerror("Error", error_msg)
        finally:
            conn.close()

    def _buscar_por_secuencia(self, _event=None) -> None:
        """Buscar elementos por número de secuencia y mostrar lista seleccionable."""
        sec = self.entry_secuencia.get().strip()

        if not sec.isdigit():
            messagebox.showwarning("Inválido", "Secuencia debe ser numérica.")
            return

        conn = self._conectar_bd()
        if not conn:
            return

        try:
            with conn.cursor() as cur:
                cur.execute(
                    sql.SQL(
                        """
                        SELECT
                            secuencia,              -- 0
                            codigo_barras,          -- 1
                            serie,                  -- 2
                            placa,                  -- 3
                            descripcion_elemento,   -- 4
                            tipo_parte,             -- 5
                            estado_elemento,        -- 6
                            ubicacion_2025,         -- 7
                            responsable_inventario  -- 8
                        FROM inventario_verificado
                        WHERE secuencia = %s
                        ORDER BY codigo_barras
                        """
                    ),
                    (sec,),
                )
                resultados = cur.fetchall()

                # Limpiar panel
                for widget in self.frame_seq.winfo_children():
                    widget.destroy()

                if resultados:
                    for r in resultados:
                        info_frame = ttk.Frame(self.frame_seq, relief="solid", padding=5)
                        info_frame.pack(fill="x", pady=2, padx=2)

                        info_text = (
                            f"Secuencia: {r[0]}\n"
                            f"Código: {r[1]}\n"
                            f"Serie: {r[2]}\n"
                            f"Placa: {r[3]}\n"
                            f"Descripción: {r[4]}\n"
                            f"Parte: {r[5]}\n"
                            f"Estado: {self._estado_to_ui(r[6])}\n"
                            f"Ubicación: {r[7]}\n"
                            f"Responsable: {r[8]}"
                        )
                        ttk.Label(
                            info_frame, text=info_text, anchor="w", justify="left"
                        ).pack(fill="x")

                        ttk.Button(
                            info_frame,
                            text="Cargar este registro",
                            command=lambda r=r: self._cargar_desde_secuencia(r)
                        ).pack(pady=2)
                else:
                    ttk.Label(self.frame_seq, text="No se encontraron registros.").pack()

                logging.info(f"Búsqueda por secuencia: {sec} - {len(resultados)} resultados")

        except Exception as e:
            error_msg = f"Error en búsqueda por secuencia: {str(e)}"
            logging.error(error_msg)
            messagebox.showerror("Error", error_msg)
        finally:
            conn.close()

    def _cargar_desde_secuencia(self, registro: tuple) -> None:
        """Cargar un registro específico desde los resultados de secuencia."""
        self.entry_busqueda.delete(0, tk.END)
        self.entry_busqueda.insert(0, registro[1])  # codigo_barras
        self._buscar_codigo()

    # =========================
    # Informe (Treeview)
    # =========================
    def _actualizar_tree_unique(
        self,
        *,
        cbarras, sec, ser, pla, desc, parte, est, ubic, resp, alm
    ) -> None:
        """
        Mantiene UNA sola fila por código de barras en el Treeview.
        Si ya existe, se ACTUALIZA en lugar de insertar otra fila.
        """
        values_tuple = (cbarras, sec, ser, (pla or "N/A"), desc, parte, est, ubic, resp, alm)

        if cbarras in self.rows_by_barcode:
            iid = self.rows_by_barcode[cbarras]
            self.tree.item(iid, values=values_tuple)
        else:
            iid = self.tree.insert("", tk.END, values=values_tuple)
            self.rows_by_barcode[cbarras] = iid

    # =========================
    # Guardar cambios BD
    # =========================
    def _guardar_bd(self) -> None:
        """Insertar o actualizar el registro actual en la base de datos (robusto)."""
        datos = {
            "centro_operacion": self.centro_operacion_entry.get().strip(),
            "secuencia": self.secuencia_entry.get().strip(),
            "codigo_barras": self.codigo_barras_entry.get().strip(),
            "codigo_elemento": self.codigo_elemento_entry.get().strip(),
            "serie": self.serie_entry.get().strip(),
            "placa": self.placa_entry.get().strip(),
            "descripcion_elemento": self.descripcion_entry.get().strip(),
            "tipo_parte": self.tipo_parte_entry.get().strip(),
            "tipo_almacenamiento": self.almacen_entry.get().strip(),
            "fecha_ingreso": self.fecha_ingreso_entry.get().strip(),
            "costo": self.costo_entry.get().strip(),
            "cedula_resp": self.cedula_combo.get().strip(),
            "responsable_inventario": self.responsable_combo.get().strip(),
            "estado_elemento": self.estado_combo.get().strip(),
            "ubicacion_2025": self.ubicacion_combo.get().strip(),
        }

        # Validaciones mínimas
        if not datos["codigo_barras"]:
            messagebox.showwarning("Faltan datos", "Código de barras es obligatorio.")
            self.codigo_barras_entry.focus()
            return

        if not datos["responsable_inventario"]:
            messagebox.showwarning("Faltan datos", "Responsable de inventario es obligatorio.")
            self.responsable_combo.focus()
            return

        # Normalizar estado al dominio permitido por la BD
        datos["estado_elemento"] = self._estado_to_db(datos["estado_elemento"])
        if datos["estado_elemento"] is None and self.estado_combo.get().strip():
            messagebox.showwarning("Valor inválido", "El estado debe ser 'BUENO' o 'INSERVIBLE'.")
            self.estado_combo.focus()
            return

        # Costo -> int o NULL (ignora 'N/A', vacíos, etc.)
        try:
            if datos["costo"] and datos["costo"].isdigit():
                datos["costo"] = int(datos["costo"])
            else:
                datos["costo"] = None
        except ValueError:
            messagebox.showwarning("Valor inválido", "El costo debe ser numérico entero.")
            self.costo_entry.focus()
            return

        # Secuencia y cédula -> int o NULL (si vienen vacíos)
        datos["secuencia"] = int(datos["secuencia"]) if datos["secuencia"].isdigit() else None
        datos["cedula_resp"] = int(datos["cedula_resp"]) if datos["cedula_resp"].isdigit() else None

        # Fecha -> validar formato si viene
        if datos["fecha_ingreso"]:
            try:
                datetime.strptime(datos["fecha_ingreso"], "%Y-%m-%d")
            except ValueError:
                messagebox.showwarning("Fecha inválida", "Use formato AAAA-MM-DD, ej. 2025-04-09.")
                self.fecha_ingreso_entry.focus()
                return

        conn = self._conectar_bd()
        if not conn:
            return

        try:
            with conn.cursor() as cur:
                # ¿Existe el código?
                cur.execute(
                    "SELECT 1 FROM inventario_verificado WHERE codigo_barras=%s LIMIT 1",
                    (datos["codigo_barras"],),
                )
                existe = cur.fetchone() is not None

                if existe:
                    query_upd = """
                        UPDATE inventario_verificado SET
                            centro_operacion=%(centro_operacion)s,
                            secuencia=%(secuencia)s,
                            codigo_elemento=%(codigo_elemento)s,
                            serie=%(serie)s,
                            placa=%(placa)s,
                            descripcion_elemento=%(descripcion_elemento)s,
                            tipo_parte=%(tipo_parte)s,
                            tipo_almacenamiento=%(tipo_almacenamiento)s,
                            fecha_ingreso = NULLIF(%(fecha_ingreso)s, '')::date,
                            costo=%(costo)s,
                            cedula_resp=%(cedula_resp)s,
                            responsable_inventario=%(responsable_inventario)s,
                            estado_elemento = NULLIF(%(estado_elemento)s, '')::estado_enum,
                            ubicacion_2025=%(ubicacion_2025)s,
                            verificado=TRUE
                        WHERE codigo_barras=%(codigo_barras)s
                    """
                    cur.execute(query_upd, datos)
                    logging.info(f"UPDATE rowcount={cur.rowcount}")

                    if cur.rowcount == 0:
                        messagebox.showwarning(
                            "Sin cambios",
                            "No se actualizó ninguna fila. Verifique que el código de barras coincida exactamente."
                        )
                        logging.warning("UPDATE sin filas afectadas (valores idénticos o whitespace).")

                    msg = "Registro actualizado."
                else:
                    query_ins = """
                        INSERT INTO inventario_verificado (
                            centro_operacion, secuencia, codigo_barras, codigo_elemento,
                            serie, placa, descripcion_elemento, tipo_parte,
                            tipo_almacenamiento, fecha_ingreso, costo, cedula_resp,
                            responsable_inventario, estado_elemento, ubicacion_2025, verificado
                        ) VALUES (
                            %(centro_operacion)s, %(secuencia)s, %(codigo_barras)s, %(codigo_elemento)s,
                            %(serie)s, %(placa)s, %(descripcion_elemento)s, %(tipo_parte)s,
                            %(tipo_almacenamiento)s, NULLIF(%(fecha_ingreso)s, '')::date, %(costo)s, %(cedula_resp)s,
                            %(responsable_inventario)s, NULLIF(%(estado_elemento)s, '')::estado_enum, %(ubicacion_2025)s, TRUE
                        )
                    """
                    cur.execute(query_ins, datos)
                    logging.info(f"INSERT rowcount={cur.rowcount}")
                    msg = "Registro insertado."

            conn.commit()
            logging.info("Commit OK")

            # Reflejar en el informe único por código (incluye Parte)
            self._actualizar_tree_unique(
                cbarras=datos["codigo_barras"],
                sec=datos["secuencia"] if datos["secuencia"] is not None else "",
                ser=datos["serie"],
                pla=datos["placa"],
                desc=datos["descripcion_elemento"],
                parte=datos["tipo_parte"],
                est=self._estado_to_ui(datos["estado_elemento"]),
                ubic=datos["ubicacion_2025"],
                resp=datos["responsable_inventario"],
                alm=datos["tipo_almacenamiento"],
            )

            messagebox.showinfo("Éxito", msg)
            logging.info(f"Registro guardado: {datos['codigo_barras']} - {msg}")

        except Exception as e:
            conn.rollback()
            error_msg = f"Error al guardar: {str(e)}"
            logging.error(error_msg)
            messagebox.showerror("Error al guardar", error_msg)
        finally:
            conn.close()

    # =========================
    # Eventos de teclado
    # =========================
    def _setup_eventos_teclado(self) -> None:
        """Configurar atajos de teclado y disparadores de búsqueda."""
        # Soporte escáner (muchos envían Enter al final)
        self.entry_busqueda.bind("<Return>", self._buscar_codigo)
        # Soporte digitación manual con debounce
        self.entry_busqueda.bind("<KeyRelease>", self._debounced_keyrelease)

        self.root.bind("<F1>", lambda _e: self._buscar_codigo())
        self.root.bind("<F2>", lambda _e: self._buscar_por_secuencia())
        self.root.bind("<F5>", lambda _e: self._guardar_bd())
        self.root.bind("<F9>", lambda _e: self._limpiar_campos())
        self.root.bind("<F12>", lambda _e: self._exportar_excel())
        self.root.bind("<Escape>", lambda _e: self._limpiar_campos())

        # Enter en campo de secuencia para buscar
        self.entry_secuencia.bind("<Return>", self._buscar_por_secuencia)

    # =========================
    # UI
    # =========================
    def _setup_ui(self) -> None:
        """Configurar la interfaz de usuario."""
        # Grid principal
        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(2, weight=1)

        # Header
        header = ttk.Frame(self.root, padding=5)
        header.grid(row=0, column=0, sticky="ew")
        header.columnconfigure(1, weight=1)
        header.columnconfigure(3, weight=1)

        ttk.Label(header, text="Buscar Código (mín. 6 díg.):").grid(row=0, column=0, sticky="w")
        self.entry_busqueda = ttk.Entry(header, width=30)
        self.entry_busqueda.grid(row=0, column=1, sticky="ew", padx=5)

        ttk.Label(header, text="Buscar por Secuencia:").grid(row=0, column=2, sticky="e", padx=(20, 5))
        self.entry_secuencia = ttk.Entry(header, width=20)
        self.entry_secuencia.grid(row=0, column=3, sticky="ew")

        ttk.Button(header, text="Buscar (F2)", command=self._buscar_por_secuencia).grid(row=0, column=4, padx=5)

        # Main content
        main = ttk.Frame(self.root, padding=10)
        main.grid(row=1, column=0, sticky="nsew")
        main.columnconfigure(0, weight=1)
        main.columnconfigure(1, weight=1)
        main.rowconfigure(0, weight=1)

        # Formulario izquierda
        form = ttk.LabelFrame(main, text="Datos del Elemento", padding=10)
        form.grid(row=0, column=0, sticky="nsew", padx=(0, 10))
        form.columnconfigure(1, weight=1)

        labels = [
            "Centro Operación", "Secuencia", "Código Barras", "Código Elemento",
            "Serie", "Placa", "Descripción", "Tipo Parte", "Almacén (Tipo)",
            "Fecha Ingreso", "Costo"
        ]

        self.all_entries: list[ttk.Entry] = []
        for i, lbl in enumerate(labels):
            ttk.Label(form, text=lbl + ":").grid(row=i, column=0, sticky="w", pady=2)
            ent = ttk.Entry(form, width=50)
            ent.grid(row=i, column=1, sticky="ew", pady=2)
            self.all_entries.append(ent)

        (
            self.centro_operacion_entry, self.secuencia_entry, self.codigo_barras_entry,
            self.codigo_elemento_entry, self.serie_entry, self.placa_entry,
            self.descripcion_entry, self.tipo_parte_entry, self.almacen_entry,
            self.fecha_ingreso_entry, self.costo_entry
        ) = self.all_entries

        # Almacén (Tipo) en readonly; se carga controlando el estado
        self.almacen_entry.config(state="readonly")

        # Comboboxes
        ttk.Label(form, text="Cédula Responsable:").grid(row=11, column=0, sticky="w")
        self.cedula_combo = ttk.Combobox(form, width=47, state="readonly")
        self.cedula_combo.grid(row=11, column=1, pady=2)

        ttk.Label(form, text="Responsable Inventario:").grid(row=12, column=0, sticky="w")
        self.responsable_combo = ttk.Combobox(form, width=47, state="readonly")
        self.responsable_combo.grid(row=12, column=1, pady=2)

        ttk.Label(form, text="Estado:").grid(row=13, column=0, sticky="w")
        self.estado_combo = ttk.Combobox(
            form,
            values=["BUENO", "INSERVIBLE"],
            width=47,
            state="readonly"
        )
        self.estado_combo.grid(row=13, column=1, pady=2)

        ttk.Label(form, text="Ubicación 2025:").grid(row=14, column=0, sticky="w")
        self.ubicacion_combo = ttk.Combobox(form, width=47, state="readonly")
        self.ubicacion_combo.grid(row=14, column=1, pady=2)

        # Botones
        btns = ttk.Frame(form)
        btns.grid(row=15, column=0, columnspan=2, pady=10)
        ttk.Button(btns, text="Guardar Cambios (F5)", command=self._guardar_bd).pack(side="left", padx=5)
        ttk.Button(btns, text="Limpiar Campos (F9)", command=self._limpiar_campos).pack(side="left", padx=5)
        ttk.Button(btns, text="Exportar Excel (F12)", command=self._exportar_excel).pack(side="left", padx=5)

        # Panel derecho para resultados por secuencia
        self.frame_seq = ttk.LabelFrame(main, text="Resultados de Búsqueda por Secuencia", relief="sunken", padding=10)
        self.frame_seq.grid(row=0, column=1, sticky="nsew")
        self.frame_seq.columnconfigure(0, weight=1)

        # Tabla inferior (informe) — incluye "Parte" junto a "Descripción"
        self.columnas = (
            "Código Barras", "Secuencia", "Serie", "Placa", "Descripción", "Parte",
            "Estado", "Ubicación 2025", "Responsable", "Almacén"
        )

        tree_frame = ttk.Frame(self.root)
        tree_frame.grid(row=2, column=0, sticky="nsew", padx=10, pady=10)
        tree_frame.columnconfigure(0, weight=1)
        tree_frame.rowconfigure(0, weight=1)

        self.tree = ttk.Treeview(tree_frame, columns=self.columnas, show="headings", height=10)

        v_scrollbar = ttk.Scrollbar(tree_frame, orient="vertical", command=self.tree.yview)
        h_scrollbar = ttk.Scrollbar(tree_frame, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set)

        self.tree.grid(row=0, column=0, sticky="nsew")
        v_scrollbar.grid(row=0, column=1, sticky="ns")
        h_scrollbar.grid(row=1, column=0, sticky="ew")

        for col in self.columnas:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100, minwidth=50)

        self.tree.column("Código Barras", width=130)
        self.tree.column("Descripción", width=260)
        self.tree.column("Parte", width=110)
        self.tree.column("Ubicación 2025", width=150)

        # Footer
        footer = ttk.Frame(self.root)
        footer.grid(row=3, column=0, sticky="ew", pady=5)
        footer.columnconfigure(0, weight=1)

        ttk.Label(
            footer,
            text="© 2024 Jorge A. Melo - F1: buscar • Enter: escáner • F2: por secuencia • F5: guardar • F9/Esc: limpiar • F12: exportar",
            font=("Segoe UI", 8)
        ).grid(row=0, column=0)

        # Foco inicial
        self.entry_busqueda.focus()


# =========================
# Punto de entrada
# =========================
if __name__ == "__main__":
    root = tk.Tk()
    app = InventarioApp(root)
    root.mainloop()